In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import torch

# Importing libraries
import copy
import os

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset
from typing import Tuple, List
from torch.optim import Optimizer

from nltk.corpus import words, wordnet
import nltk

import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

BASE_PATH = 'drive/MyDrive/NLP_project/'
# BASE_PATH = './'
if not os.path.exists(BASE_PATH):
    raise ValueError('path does not exist')

In [4]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def skip(line, cell):
    return

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cpu


# Load dataset

In [6]:
df_train = pd.read_csv(BASE_PATH + 'datasets/EmoTrain.csv')
df_test = pd.read_csv(BASE_PATH + 'datasets/EmoVal.csv')

In [7]:
df_train = df_train.drop(axis=1, columns=['Unnamed: 0'])
df_test = df_test.drop(axis=1, columns=['Unnamed: 0'])
df_train

,text,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,Is there some scripture you could quote me? I'...,1,0,0,0,0,1,0,0,0,0
1,Good. Now we just need people to dislike commi...,1,0,0,0,0,0,0,0,0,0
2,This was driving me NUTS!,0,1,0,0,0,0,0,0,0,0
3,Thank you for your advice!,0,0,0,0,0,0,0,0,0,1
4,Some do. Some don't. Blanket generalizations a...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
190097,They needed to insert the phrase “over mainten...,0,0,0,0,1,0,0,0,0,0
190098,Back in the seventies and eighties we all did ...,1,0,0,0,0,0,0,0,0,0
190099,"6lbs is a lap dog, if someone shoots that caus...",0,0,0,0,0,0,0,1,0,0
190100,This gets much worse on the 2nd loop.,0,0,0,0,1,0,0,0,0,0


In [8]:
# step = 5_000
df_train = df_train[:150_000]
df_test = df_test[:150_000]

# Pre processing

## Description of data

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   text         150000 non-null  object
 1   praise       150000 non-null  int64 
 2   amusement    150000 non-null  int64 
 3   anger        150000 non-null  int64 
 4   disapproval  150000 non-null  int64 
 5   confusion    150000 non-null  int64 
 6   interest     150000 non-null  int64 
 7   sadness      150000 non-null  int64 
 8   fear         150000 non-null  int64 
 9   joy          150000 non-null  int64 
 10  love         150000 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 12.6+ MB


In [10]:
print(df_train.isnull().sum())

text           0
praise         0
amusement      0
anger          0
disapproval    0
confusion      0
interest       0
sadness        0
fear           0
joy            0
love           0
dtype: int64


In [11]:
def split_features_labels(df) -> Tuple[pd.Series, pd.DataFrame]:
    x = df['text']
    y = df.drop(axis=1, columns=['text'])
    return x, y


x_train, y_train = split_features_labels(df_train)
x_test, y_test = split_features_labels(df_test)

In [12]:
x_train

,text
0,Is there some scripture you could quote me? I'...
1,Good. Now we just need people to dislike commi...
2,This was driving me NUTS!
3,Thank you for your advice!
4,Some do. Some don't. Blanket generalizations a...
...,...
149995,"Yes, some doctors look down on military docs. ..."
149996,"for how long, you arent fucking with me?"
149997,"You shouldn’t forget it, and you probably need..."
149998,Life Hack Attack has become Attack Everyone's ...


In [13]:
y_train

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
0,1,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1
4,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
149995,1,0,0,0,0,0,0,0,0,0
149996,0,0,1,0,0,0,0,0,0,0
149997,0,0,0,0,0,0,0,0,0,1
149998,0,0,0,0,0,0,0,0,0,1


## Data cleaning

Lower case all the words

In [14]:
def lower_case(x):
    x = x.str.lower()
    return x

x_train = lower_case(x_train)
x_test = lower_case(x_test)

In [15]:
x_train

,text
0,is there some scripture you could quote me? i'...
1,good. now we just need people to dislike commi...
2,this was driving me nuts!
3,thank you for your advice!
4,some do. some don't. blanket generalizations a...
...,...
149995,"yes, some doctors look down on military docs. ..."
149996,"for how long, you arent fucking with me?"
149997,"you shouldn’t forget it, and you probably need..."
149998,life hack attack has become attack everyone's ...


Convert to tokens

In [16]:
x_train = x_train.to_list()
x_test = x_test.to_list()

In [17]:
print(len(x_train))
print(x_train[:5])

150000
["is there some scripture you could quote me? i'd like to read up on it just to be sure for myself", 'good. now we just need people to dislike commies more than they do now', 'this was driving me nuts!', 'thank you for your advice!', "some do. some don't. blanket generalizations are almost always false and unhelpful."]


In [18]:
def list_of_words(x):
    x = list(map(lambda i: i.split(), x))
    return x

x_train = list_of_words(x_train)
x_test = list_of_words(x_test)

Then we have **list** of **list** of **tokens**

In [19]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me?',
  "i'd",
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself'],
 ['good.',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts!'],
 ['thank', 'you', 'for', 'your', 'advice!'],
 ['some',
  'do.',
  'some',
  "don't.",
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful.'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size.'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to.',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article.',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  "i'm",
  'feeling',
  'pumped',
  'about',
  'start

What kind of **characters** are there in the dataset?

In [20]:
characters = {'isalnum': 0}
for text in x_train:
    for word in text:
        for c in word:
            if c.isalnum():
                characters['isalnum'] += 1
            elif c in characters:
                characters[c] += 1
            else:
                characters[c] = 0

In [21]:
print(len(characters))
keys = list(characters.keys())
keys.sort()
print(keys)
characters

463
['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'isalnum', '{', '|', '}', '~', '¡', '£', '¦', '©', '«', '¬', '¯', '°', '´', '·', '»', '̕', '̖', '̗', '̘', '̙', '̜', '̝', '̞', '̟', '̠', '̢', '̣', '̤', '̥', '̦', '̧', '̨', '̪', '̫', '̬', '̭', '̮', '̯', '̰', '̱', '̲', '̳', '̶', '̷', '̹', '̺', '̻', '̼', 'ͅ', '͈', '͍', '͎', '͏', '͓', '͔', '͕', '͖', '͘', '͙', '͚', '͜', '͝', '͞', '͟', '͠', '͡', '͢', '׳', '\u200d', '–', '—', '‘', '’', '“', '”', '„', '•', '…', '\u202a', '\u202c', '‽', '€', '™', '√', '≈', '≠', '▀', '▫', '☁', '☂', '☆', '☕', '☝', '☠', '☹', '☺', '♀', '♂', '♡', '♥', '♪', '♫', '♭', '♾', '♿', '⚔', '⚡', '⚰', '⛏', '⛑', '✊', '✋', '✌', '✔', '✨', '❄', '❣', '❤', '⠀', '⠁', '⠃', '⠄', '⠇', '⠈', '⠉', '⠊', '⠋', '⠏', '⠓', '⠘', '⠙', '⠚', '⠛', '⠞', '⠟', '⠢', '⠦', '⠫', '⠳', '⠴', '⠶', '⠸', '⠹', '⠻', '⠾', '⠿', '⡀', '⡄', '⡆', '⡇', '⡏', '⡜', '⡟', '⡶', '⡷', '⡼', '⡾', '⡿', '⢀', '⢠', '⢣', '⢤', '⢧', '⢰', '⢸', '⢹'

{'isalnum': 8147859,
 '?': 18465,
 "'": 51903,
 '.': 163588,
 '!': 29626,
 '’': 23468,
 '[': 26800,
 ']': 26786,
 ',': 49651,
 '"': 8509,
 ':': 5366,
 ')': 4102,
 '-': 6852,
 '💩': 2,
 '/': 4382,
 '*': 6842,
 '(': 3039,
 '^': 1077,
 '>': 2387,
 '~': 794,
 '”': 1736,
 '😒': 16,
 '£': 36,
 '“': 1748,
 '🙄': 65,
 '🍻': 6,
 '—': 84,
 '😂': 915,
 '=': 324,
 '$': 591,
 '😍': 84,
 '😪': 9,
 ';': 693,
 '&': 278,
 '😭': 161,
 '❤': 275,
 '️': 566,
 '%': 741,
 '͡': 35,
 '͜': 23,
 '🅱': 24,
 '😔': 38,
 '+': 387,
 '🤕': 7,
 '@': 98,
 '🤣': 258,
 '#': 610,
 '🙌': 38,
 '😇': 8,
 '🎶': 40,
 '💃': 21,
 '<': 385,
 '\u200d': 177,
 '♀': 110,
 '☺': 49,
 '😢': 64,
 '😦': 1,
 '😜': 11,
 '♡': 2,
 '_': 509,
 '‘': 252,
 '💪': 18,
 '🤔': 126,
 '😣': 7,
 '😊': 71,
 '🐢': 2,
 '😄': 24,
 '😐': 23,
 '😤': 267,
 '😞': 11,
 '¦': 2,
 '¯': 67,
 '😱': 20,
 '👍': 100,
 '🍑': 2,
 '😹': 2,
 '👌': 69,
 '🏾': 8,
 '😕': 12,
 '♂': 63,
 '🏻': 104,
 '–': 9,
 '💕': 30,
 '❣': 9,
 '😎': 68,
 '😡': 37,
 '🤬': 11,
 '😴': 5,
 '€': 24,
 '👏': 147,
 '™': 23,
 '🤢': 42,
 '🕺': 2,
 

- Remove non-semantic characters
- Convert `?` and `!` and `emojis` to tokens

In [22]:
def split_special_chars(word: str):
    if word.isalnum():
        return word, []
    specials = list(filter(lambda c: c == '!' or c == '?', word))
    return ''.join(filter(lambda c: c.isalnum(), word)), specials

def clean_tokens(x):
    for text in tqdm(x):
        specials = []
        for i, word in enumerate(text):
            text[i], special_word = split_special_chars(word)
            specials.extend(special_word)
        text.extend(specials)

clean_tokens(x_train)
clean_tokens(x_test)

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

In [23]:
x_train

[['is',
  'there',
  'some',
  'scripture',
  'you',
  'could',
  'quote',
  'me',
  'id',
  'like',
  'to',
  'read',
  'up',
  'on',
  'it',
  'just',
  'to',
  'be',
  'sure',
  'for',
  'myself',
  '?'],
 ['good',
  'now',
  'we',
  'just',
  'need',
  'people',
  'to',
  'dislike',
  'commies',
  'more',
  'than',
  'they',
  'do',
  'now'],
 ['this', 'was', 'driving', 'me', 'nuts', '!'],
 ['thank', 'you', 'for', 'your', 'advice', '!'],
 ['some',
  'do',
  'some',
  'dont',
  'blanket',
  'generalizations',
  'are',
  'almost',
  'always',
  'false',
  'and',
  'unhelpful'],
 ['those', 'are', 'separate', 'issues', 'from', 'the', 'sample', 'size'],
 ['my',
  'fur',
  'son',
  'was',
  'the',
  'first',
  'i',
  'came',
  'out',
  'to',
  'most',
  'important',
  'also'],
 ['really',
  'appreciate',
  'this',
  'post',
  'and',
  'article',
  'i',
  'just',
  'subscribed',
  'and',
  'between',
  'this',
  'and',
  'the',
  'sidebar',
  'im',
  'feeling',
  'pumped',
  'about',
  's

Remove stop-words

In [24]:
print(len(ENGLISH_STOP_WORDS))
print(ENGLISH_STOP_WORDS)

318
frozenset({'ltd', 'is', 'so', 'become', 'throughout', 'elsewhere', 'neither', 'may', 'being', 'either', 'whenever', 'last', 'has', 'sometimes', 'up', 'because', 'whoever', 'one', 'hers', 'almost', 'whereupon', 'during', 'her', 'myself', 'he', 'moreover', 'must', 'something', 'wherein', 'cry', 'within', 'nor', 'done', 'in', 'whereafter', 'yours', 'have', 'put', 'i', 'a', 'bottom', 'hereafter', 'beforehand', 'him', 'hence', 'to', 'of', 'anyway', 'across', 'with', 'together', 'can', 'amoungst', 'three', 'inc', 'sixty', 'us', 'along', 'alone', 'when', 'seeming', 'thin', 'very', 'hundred', 'should', 'if', 'be', 'before', 'few', 'on', 'becoming', 'while', 'un', 'around', 'ours', 'nobody', 'out', 'wherever', 'it', 'nowhere', 'not', 'all', 'now', 'by', 'she', 'were', 'under', 'meanwhile', 'indeed', 'an', 'ourselves', 'otherwise', 'mine', 'many', 'bill', 'hasnt', 'why', 'beyond', 'made', 'and', 'here', 'becomes', 'namely', 'this', 'me', 'itself', 'everything', 'sometime', 'less', 'my', 'fil

In [25]:
def remove_stop_words(x):
    x = list(map(lambda text: list(filter(lambda word: word not in ENGLISH_STOP_WORDS, text)), x))
    return x

x_train = remove_stop_words(x_train)
x_test = remove_stop_words(x_test)

# Vectorize the dataset

An example of vectorization

In [26]:
df = pd.DataFrame(dtype=bool)
for i, text in enumerate(tqdm(x_train[:5])):
    for word in text:
        df.loc[i, word] = True
df.fillna(False, inplace=True)

  0%|          | 0/5 [00:00<?, ?it/s]

<ipython-input-26-c6cb43738d2c>:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(False, inplace=True)


In [27]:
df

,scripture,quote,id,like,read,just,sure,?,good,need,...,driving,nuts,!,thank,advice,dont,blanket,generalizations,false,unhelpful
0,True,True,True,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True


In [28]:
top_words = {}
for text in x_train:
    for word in text:
            if word in top_words:
                top_words[word] += 1
            else:
                top_words[word] = 1

top_words_ordered = sorted(top_words.items(), key=lambda kv: kv[1])
print(top_words_ordered)

[('frend', 1), ('actally', 1), ('representations', 1), ('1911', 1), ('rummages', 1), ('foals', 1), ('50100', 1), ('rwoosh', 1), ('welcher', 1), ('spitballing', 1), ('tpss', 1), ('antifascist', 1), ('itation', 1), ('eeded', 1), ('molon', 1), ('labe', 1), ('biblethump', 1), ('pita', 1), ('skynet', 1), ('ubeelzebambi', 1), ('qualifiers', 1), ('snowbirds', 1), ('iwreck', 1), ('commemorating', 1), ('anuses', 1), ('instructive', 1), ('upvoters', 1), ('whatss', 1), ('2lbs', 1), ('pussycat', 1), ('seventh', 1), ('sadbut', 1), ('comatose', 1), ('sedation', 1), ('exorbitantly', 1), ('otw', 1), ('4055', 1), ('primates', 1), ('steakhouse', 1), ('yepand', 1), ('notoriety', 1), ('rl', 1), ('中国人', 1), ('usamuelzeign', 1), ('collegehumor', 1), ('vase', 1), ('resetting', 1), ('gooood', 1), ('staking', 1), ('carpenter', 1), ('soyou', 1), ('standoffs', 1), ('ruplifting', 1), ('eternally', 1), ('thrussy', 1), ('refrigerated', 1), ('livable', 1), ('vita', 1), ('zeroh', 1), ('betta', 1), ('wary', 1), ('conq

In [29]:
print(list(map(lambda x: x[1], top_words_ordered)).index(1))
print(list(map(lambda x: x[1], top_words_ordered)).index(2))
print(list(map(lambda x: x[1], top_words_ordered)).index(3))
print(list(map(lambda x: x[1], top_words_ordered)).index(4))
print(list(map(lambda x: x[1], top_words_ordered)).index(5))

0
2223
8109
14414
17611


In [30]:
word_list = list(map(lambda x: x[0], top_words_ordered[list(map(lambda x: x[1], top_words_ordered)).index(3):]))
print(len(word_list))

24472


Create an index for every unique word

In [31]:
tensor_index = {}
for i, word in enumerate(tqdm(word_list)):
    tensor_index[word] = i

  0%|          | 0/24472 [00:00<?, ?it/s]

In [32]:
print(len(tensor_index))

24472


Check synonyms

In [33]:
nltk.download('wordnet')  # Download WordNet if not already installed

def get_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name().lower())
    return synonyms - {word}

word = "happy"
print(get_synonyms(word))

[nltk_data] Downloading package wordnet to /root/nltk_data...


{'glad', 'felicitous', 'well-chosen'}


Create a tensor

In [41]:
def create_x_tensor(x, syn=False):
    x_tensor = torch.zeros(len(x), len(tensor_index), dtype=torch.float16)
    for i, text in enumerate(tqdm(x)):
        for word in text:
            if word in tensor_index:
                x_tensor[i, tensor_index[word]] += 1
                if syn:
                    for syn_word in get_synonyms(word):
                        if syn_word in tensor_index:
                            x_tensor[i, tensor_index[syn_word]] += 1
    return x_tensor

In [35]:
# x_train
# last_index = 0
# for i in range(0, len(x_train), step):
#     last_index = i
#     torch.save(create_x_tensor(x_train[i:i+step]), BASE_PATH + f'tensors/x_train_tensor{i//step}.pt')
# torch.save(create_x_tensor(x_train[last_index:]), BASE_PATH + f'tensors/x_train_tensor{last_index//step + 1}.pt')

In [36]:
torch.save(create_x_tensor(x_train, syn=True), BASE_PATH + 'tensors/x_train_tensor.pt')
torch.save(create_x_tensor(x_test), BASE_PATH + 'tensors/x_test_tensor.pt')

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/10562 [00:00<?, ?it/s]

Convert labels to tensor

In [37]:
y_train.describe()

,praise,amusement,anger,disapproval,confusion,interest,sadness,fear,joy,love
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,0.157893,0.069707,0.095260,0.076940,0.334427,0.063813,0.080607,0.021293,0.127633,0.117693
std,0.364642,0.254653,0.293575,0.266497,0.471791,0.244421,0.272231,0.144361,0.333682,0.322246
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
def create_y_tensor(y):
    y_tensor = torch.tensor(y.values, dtype=torch.float16)
    return y_tensor

In [39]:
# y_train
# last_index = 0
# for i in range(0, len(y_train), step):
#     last_index = i
#     torch.save(create_y_tensor(y_train[i:i+step]), BASE_PATH + f'tensors/y_train_tensor{i//step}.pt')
# torch.save(create_y_tensor(y_train[last_index:]), BASE_PATH + f'tensors/y_train_tensor{last_index//step + 1}.pt')

In [40]:
torch.save(create_y_tensor(y_train), BASE_PATH + 'tensors/y_train_tensor.pt')
torch.save(create_y_tensor(y_test), BASE_PATH + 'tensors/y_test_tensor.pt')